In [1]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf as pd
import cudf
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="0")
client = Client(cluster)
client.run(cudf.set_allocator, "managed")
client

distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 188.59 GiB
Comm: tcp://127.0.0.1:45069,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 188.59 GiB
Comm: tcp://127.0.0.1:38483,Total threads: 1
Dashboard: http://127.0.0.1:40423/status,Memory: 188.59 GiB
Nanny: tcp://127.0.0.1:41477,


In [2]:
mrdef_columns = ['CUI','AUI','ATUI','SATUI','SAB','DEF','SUPPRESS','CVF']
mrhier_columns = ['CUI','REL_AUI','CXN','PAUI','SAB','HIER_RELA','PTR','HCD','CVF']
mrsty_columns = ['CUI','TUI','STN','STY','ATUI','CVF']
mrrel_columns = ['CUI1','AUI1','STYPE1','REL','CUI2','AUI2','STYPE2','RELA','RUI','SRUI','SAB','SL','RG','DIR','SUPPRESS','CVF']
mrsat_columns = ['CUI','LUI','SUI','METAUI','STYPE','CODE','ATUI','SATUI','ATN','SAB','ATV','SUPPRESS','CVF']
mrconso_columns = ['CUI','LAT','TS','LUI','STT','SUI','ISPREF','AUI','SAUI','SCUI','SDUI','SAB','TTY','CODE','STR','SRL','SUPPRESS','CVF']
mrconso_columns2 = ['CUI','LAT','TS','LUI','STT','SUI','ISPREF','AUI2','SAUI','SCUI','SDUI','SAB','TTY','CODE','STR','SRL','SUPPRESS','CVF']


In [3]:
#1. Find all atoms of a UMLS concepts
mrconso_iter2 = pd.read_csv(
    "/home/karl/PycharmProjects/doctorlingo/test_scripts/cwi-master/CWI_Sequence_Labeller/datasets/UMLS/MRCONSO.RRF",
    sep="|",
    names=mrconso_columns,
    #chunksize="1 GB"
)
mrconso_iter2

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
npartitions=8,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,float64,object,object,object,object,object,object,int64,object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
mrconso2 = mrconso_iter2[mrconso_iter2["LAT"] == "ENG"] # select all english terms
mrconso2 = mrconso_iter2[mrconso_iter2["ISPREF"] == "Y"] # select only preferred term strings for AUIs
# mrconso = mrconso.map_partitions(lambda df: df.to_pandas())
mrconso2 = mrconso2[['CUI','STR','AUI']].compute()
mrconso2

,CUI,STR,AUI
0,C0000005,(131)I-Macroaggregated Albumin,A26634265
1,C0000005,(131)I-MAA,A26634266
2,C0000005,Macroagrégats d'albumine marquée à l'iode 131,A13433185
3,C0000005,MAA-I 131,A27488794
4,C0000005,Macroagrégats d'albumine humaine marquée à l'i...,A27614225
...,...,...,...
1351041,C5441501,Unacceptable PDX,A32650879
1351042,C5441502,Unacceptable DX1,A32650952
1351043,C5441503,"Musculoskeletal pain, not low back pain",A32650646
1351044,C5441504,"Medication Reference Terminology, 2021_03_01",A32651413


In [5]:
mrconso2['AUI+STR'] = mrconso2['STR'] +'**'+mrconso2['AUI']
mrconso2

,CUI,STR,AUI,AUI+STR
0,C0000005,(131)I-Macroaggregated Albumin,A26634265,(131)I-Macroaggregated Albumin**A26634265
1,C0000005,(131)I-MAA,A26634266,(131)I-MAA**A26634266
2,C0000005,Macroagrégats d'albumine marquée à l'iode 131,A13433185,Macroagrégats d'albumine marquée à l'iode 131*...
3,C0000005,MAA-I 131,A27488794,MAA-I 131**A27488794
4,C0000005,Macroagrégats d'albumine humaine marquée à l'i...,A27614225,Macroagrégats d'albumine humaine marquée à l'i...
...,...,...,...,...
1351041,C5441501,Unacceptable PDX,A32650879,Unacceptable PDX**A32650879
1351042,C5441502,Unacceptable DX1,A32650952,Unacceptable DX1**A32650952
1351043,C5441503,"Musculoskeletal pain, not low back pain",A32650646,"Musculoskeletal pain, not low back pain**A3265..."
1351044,C5441504,"Medication Reference Terminology, 2021_03_01",A32651413,"Medication Reference Terminology, 2021_03_01**..."


In [6]:
mrconso2 = mrconso2.drop(['STR', 'AUI'], axis=1)
mrconso2

,CUI,AUI+STR
0,C0000005,(131)I-Macroaggregated Albumin**A26634265
1,C0000005,(131)I-MAA**A26634266
2,C0000005,Macroagrégats d'albumine marquée à l'iode 131*...
3,C0000005,MAA-I 131**A27488794
4,C0000005,Macroagrégats d'albumine humaine marquée à l'i...
...,...,...
1351041,C5441501,Unacceptable PDX**A32650879
1351042,C5441502,Unacceptable DX1**A32650952
1351043,C5441503,"Musculoskeletal pain, not low back pain**A3265..."
1351044,C5441504,"Medication Reference Terminology, 2021_03_01**..."


In [7]:
mrconso2 = mrconso2.groupby(['CUI']).agg('unique').reset_index()
mrconso = pd.from_cudf(mrconso2,npartitions=1)
mrconso = mrconso.compute().to_pandas()
mrconso['AUI+STR'] = mrconso['AUI+STR'].astype('object')
mrconso = pd.from_cudf(mrconso,npartitions=1)
mrconso


,CUI,AUI+STR
npartitions=1,,
0,object,object
4431983,...,...


In [35]:
# 2. Find all source contexts associated with a UMLS concept.
# AKA find all relationships for a concept that are intra-source
# The sources are where UMLS got the data from and we extract relationship hierarchies like "is a"
mrhier = pd.read_csv(
    "/home/karl/PycharmProjects/doctorlingo/test_scripts/cwi-master/CWI_Sequence_Labeller/datasets/UMLS/MRHIER.RRF",
    sep="|",
    names=mrhier_columns,
)
mrhier = mrhier[mrhier["SAB"] == "SNOMEDCT_US"]
mrhier

,CUI,REL_AUI,CXN,PAUI,SAB,HIER_RELA,PTR,HCD,CVF
npartitions=20,,,,,,,,,
,object,object,int64,object,object,int8,object,object,int8
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [36]:
mrhier = mrhier[['CUI','HIER_RELA','REL_AUI','PAUI','PTR']]
mrhier.compute()

,CUI,HIER_RELA,REL_AUI,PAUI,PTR
28,C0000052,isa,A27769867,A3493340,A3684559.A3206010.A29521581.A2873362.A2887534....
29,C0000052,isa,A27769867,A3493340,A3684559.A3206010.A29542572.A16967690.A2953222...
30,C0000052,isa,A27769867,A3493340,A3684559.A3206010.A29542572.A16967690.A2953222...
31,C0000052,isa,A27769867,A3106815,A3684559.A3206010.A3738093.A3248739.A3101579.A...
32,C0000052,isa,A27769867,A3493340,A3684559.A3206010.A3811138.A2873295.A2873285.A...
...,...,...,...,...,...
341103,C5441268,isa,A32649650,A3004947,A3684559.A3886745.A2880798.A6938220.A3399957.A...
341104,C5441268,isa,A32649650,A3004947,A3684559.A3886745.A2880798.A6938220.A3399957.A...
341105,C5441268,isa,A32649650,A3004947,A3684559.A3886745.A2880798.A6938220.A6938316.A...
341106,C5441268,isa,A32649650,A28932762,A3684559.A3886745.A2880798.A6938220.A6938316.A...


In [37]:
mrhier['REL_AUI_PARENTAUI_PTR'] = mrhier['HIER_RELA'].astype('object')+'**'+mrhier['REL_AUI']+'**'+mrhier['PAUI']+'**'+mrhier['PTR']
mrhier

,CUI,HIER_RELA,REL_AUI,PAUI,PTR,REL_AUI_PARENTAUI_PTR
npartitions=20,,,,,,
,object,int8,object,object,object,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [38]:
mrhier = mrhier.drop(['HIER_RELA','REL_AUI','PAUI','PTR'], axis=1)
mrhier

,CUI,REL_AUI_PARENTAUI_PTR
npartitions=20,,
,object,object
,...,...
...,...,...
,...,...
,...,...


In [39]:
mrhier = mrhier.reset_index()
mrhier = mrhier.drop(['index'], axis=1)
mrhier.to_csv('/home/karl/PycharmProjects/doctorlingo/build_corpus_manual/mrhiertemp.csv', single_file=True)

['/home/karl/PycharmProjects/doctorlingo/build_corpus_manual/mrhiertemp.csv']

In [40]:
mrconso.to_csv('/home/karl/PycharmProjects/doctorlingo/build_corpus_manual/mrconso.csv', single_file=True)

['/home/karl/PycharmProjects/doctorlingo/build_corpus_manual/mrconso.csv']

In [52]:
mrhier = pd.read_csv(
    "/home/karl/PycharmProjects/doctorlingo/build_corpus_manual/mrhiertemp.csv",
    sep=",",
    dtype='object'
)

mrconso = pd.read_csv(
    "/home/karl/PycharmProjects/doctorlingo/build_corpus_manual/mrconso.csv",
    sep=",",
    dtype='object'
)

In [51]:
#o_partitions = max(mrconso.npartitions, mrhier.npartitions) * 2
#merged_df = mrconso.merge(mrhier,npartitions=o_partitions, how="left", on="CUI")
merged_df = mrconso.merge(mrhier, how="left", on="CUI")
#merged_df = merged_df.drop_duplicates()
merged_df = merged_df.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1)#.compute().to_pandas()
merged_df#.compute().to_pandas()

,Unnamed: 0_x,CUI,AUI+STR,Unnamed: 0_y,REL_AUI_PARENTAUI_PTR
npartitions=8,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
mrrel_iter = pd.read_csv(
    "/home/karl/PycharmProjects/doctorlingo/test_scripts/cwi-master/CWI_Sequence_Labeller/datasets/UMLS/MRREL.RRF",
    sep="|",
    names=mrrel_columns,
)

mrrel = mrrel_iter[mrrel_iter["RELA"] == "isa"]
# mrrel = mrrel.map_partitions(lambda df: df.to_pandas())


In [11]:
mrrel = mrrel[["CUI", "AUI", "AUI2"]]
mrrel = mrrel.drop_duplicates()
mrrel




,CUI,AUI,AUI2
npartitions=1,,,
,object,object,object
,...,...,...


In [8]:
merged_df.to_csv('mrconso_mrrel_hypernym_first_merge.csv', single_file=True)

['/home/karl/PycharmProjects/pythonProject/buildNERhypernymy/mrconso_mrrel_hypernym_first_merge.csv']

In [ ]:

"""
SOURCE = CONCEPT

1. Find all atoms of a UMLS concept.

SELECT * FROM mrconso
WHERE cui = 'C0032344';

2. Find all source definitions associated with a UMLS concept.

SELECT * FROM mrdef
WHERE cui = 'C0032344';

3. Find all source contexts associated with a UMLS concept.
All intra-source/concept relationship
SELECT * FROM mrhier
WHERE cui = 'C0032344';

4. Find all attributes for a UMLS concept.

SELECT * FROM mrsat
WHERE cui = 'C0032344'
     AND stype = 'CUI';

5. Find all semantic types for a UMLS concept.

SELECT * FROM mrsty
WHERE cui = 'C0032344';

6.a. Find all relationships for a UMLS concept.
All inter-source/concept relationships (synonyms, etc)
Note: In MRREL, the REL/RELA always expresses the nature of the relationship from CUI2 to the "current concept", CUI1. Because we're querying CUI1 below, this represents the "natural" direction of the relationship.

SELECT * FROM mrrel
WHERE cui1 = 'C0032344'

6.b. Find all inverse relationships for a UMLS concept.
Note: In MRREL, the REL/RELA always expresses the nature of the relationship from CUI2 to the "current concept", CUI1. Because we're querying CUI2 below, this represents the opposite of the "natural" direction of the relationship.

SELECT * FROM mrrel
WHERE cui2 = 'C0032344';
     AND stype2 = 'CUI';

7. Find all relationships for a concept and the preferred (English) name of the CUI2.

SELECT a.cui1, a.cui2, b.str FROM mrrel a, mrconso b
WHERE a.cui1 = 'C0032344'
     AND a.stype1 = 'CUI'
     AND a.cui2 = b.cui
     AND b.ts = 'P'
     AND b.stt = 'PF'
     AND b.ispref = 'Y'
     AND b.lat = 'ENG';

Concatenated rows exceeds size_type range after merge operation[BUG]

Solution:

You are running into this issue because the output merge of small chunks may be too big (> 2,147,483,648) .

This can happen if you have a lot of repeated keys as we repeat each row in the right table that matches the left table.
See the below example for an explanation.

Now mitigating this can be done in multiple ways :

1. If one of the frames is a single partitions (For broadcast joints) :
Repartition the multi partitioned table upfront into more partitions:

## the right frame is a single partition
assert ddf_2.npartitions == 1

merged_df = ddf_1 = ddf_1.repartition(npartitions=ddf_1.npartitions*2)
ddf_1.merge(ddf_2,how=inner)

2. If both of them have multiple partitions (Hash-Join)
Using npartitions: This argument controls the number of output partitions.

o_partitions = max(ddf_1.npartitions, ddf_2.npartitions )*2
merged_df = ddf_1.merge(ddf_2,npartitions= o_partitions,  how=inner)

It's tough to know how to mitigate this error without more information
like the number of left and right partitions but I hope that this still helps.

https://www.gitmemory.com/issue/rapidsai/cudf/8182/834834910

cudf memory error solutions:

Try using RMM managed memory.
You can enable that by setting the RMM allocator through a Dask client.run call:
i.e: the preamble to this script

client = Client(cluster)  # Sample client connecting to `cluster` object
client.run(cudf.set_allocator, "managed")  # Uses managed memory instead of "default"
By doing this, you'll be using CUDA unified memory, which will take care of moving data GPU<->CPU automatically when the GPU is running OOM.

EDIT: If you're not using Dask, you can simply use cudf.set_allocator("managed").

https://docs.rapids.ai/api/dask-cuda/nightly/index.html
https://dask-cuda.readthedocs.io/en/latest/specializations.html#spilling-from-device
"""